In [1]:
from src.eeg_data_utils import EegData
from src.audio_data_utils import  AudioData

In [2]:
filepathEeg = '/home/owaismujtaba/BIDSSpeechDatset/rawData/F10/VCV/GOMEZCARMONA~ _67e7880a-4065-4e37-a165-58982f5f8c3c.edf'
filepathAudio = '/home/owaismujtaba/BIDSSpeechDatset/rawData/F10/VCV/sub-MartaGomezCarmona_ses-VCV_Ses01_task-Default_run-001_eeg.xdf'

In [3]:
eegData = EegData(filepathEeg)
#audioData = AudioData(filepathAudio)

*******************Loading /home/owaismujtaba/BIDSSpeechDatset/rawData/F10/VCV/GOMEZCARMONA~ _67e7880a-4065-4e37-a165-58982f5f8c3c.edf File*******************
Extracting EDF parameters from /home/owaismujtaba/BIDSSpeechDatset/rawData/F10/VCV/GOMEZCARMONA~ _67e7880a-4065-4e37-a165-58982f5f8c3c.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3055871  =      0.000 ...  5968.498 secs...
*******************Loaded /home/owaismujtaba/BIDSSpeechDatset/rawData/F10/VCV/GOMEZCARMONA~ _67e7880a-4065-4e37-a165-58982f5f8c3c.edf File*******************
> /home/owaismujtaba/BIDSSpeechDatset/src/utils.py(210)eegTransitionTriggerPoints()
    208     negTtransitionPointsIndexes = np.array([0] + differenceArray.tolist())
    209     pdb.set_trace()
--> 210     return transitionPointsIndexes
    211 
    212 def loadEdfFile(filepath):

